In [1]:
BERT_BASE_DIR = '/Users/m-suzuki/work/japanese-bert/jawiki-20190901/mecab-ipadic-bpe-32k/do-whole-word-mask'

In [2]:
import torch
from transformers import BertForMaskedLM
from tokenization import MecabBertTokenizer

I1030 22:45:41.339545 4452584896 file_utils.py:32] TensorFlow version 2.0.0 available.
I1030 22:45:41.340346 4452584896 file_utils.py:39] PyTorch version 1.3.0.post2 available.
I1030 22:45:41.590382 4452584896 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
model = BertForMaskedLM.from_pretrained(BERT_BASE_DIR)

I1030 22:45:41.636089 4452584896 configuration_utils.py:148] loading configuration file /Users/m-suzuki/work/japanese-bert/jawiki-20190901/mecab-ipadic-bpe-32k/do-whole-word-mask/config.json
I1030 22:45:41.637474 4452584896 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 32000
}

I1030 22:45:41.638771 4452584896 modeling_utils.py:334] loading weights file /Users/m-suzuki/work/japanese-bert/jawiki-20190901/mecab-ipadic-bpe-32k/do-whole-word-mask/pytorch_model.bin
I1030

In [4]:
tokenizer = MecabBertTokenizer(vocab_file=f'{BERT_BASE_DIR}/vocab.txt')

In [5]:
text = '朝食に[MASK]を食べました。'

In [6]:
token_ids = tokenizer.encode(text, add_special_tokens=True)

In [7]:
token_ids

[2, 25965, 7, 4, 11, 2949, 3913, 10, 8, 3]

In [8]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)

In [9]:
tokens

['[CLS]', '朝食', 'に', '[MASK]', 'を', '食べ', 'まし', 'た', '。', '[SEP]']

In [10]:
token_ids = torch.tensor([token_ids])

In [11]:
token_ids

tensor([[    2, 25965,     7,     4,    11,  2949,  3913,    10,     8,     3]])

In [12]:
predictions, = model(token_ids)
_, top10_pred_ids = torch.topk(predictions, k=10, dim=2)

for correct_id, pred_ids in zip(token_ids[0], top10_pred_ids[0]):
    correct_token = tokenizer.convert_ids_to_tokens([correct_id.item()])
    pred_tokens = tokenizer.convert_ids_to_tokens(pred_ids.tolist())
    print(correct_token, pred_tokens)

['[CLS]'] ['、', '」', 'まし', 'て', 'た', 'です', '朝食', 'お', '朝', '今']
['朝食'] ['朝食', '朝', '夕食', '早朝', '午後', '昼', '最初', '午前', '食事', '代わり']
['に'] ['に', 'は', 'として', 'の', '用', 'で', '2', '、', 'ニ', '後']
['[MASK]'] ['パン', '朝食', 'ワイン', 'ご飯', 'コーヒー', 'カレー', '野菜', '魚', 'サラダ', '果物']
['を'] ['を', '##食', '##イッチ', 'が', 'も', 'と', 'は', '##テト', 'の', 'カレー']
['食べ'] ['食べ', '飲み', '与え', 'し', '食い', '食べる', '待ち', '受け', '始め', '焼き']
['まし'] ['まし', 'でし', 'ます', 'ましょ', 'ませ', 'し', 'て', 'です', 'ちゃっ', '直し']
['た'] ['た', 'て', '」', 'たら', 'ます', 'まし', 'し', 'よ', 'り', 'ん']
['。'] ['。', '」', '!', '...。', 'が', '、', '......', ':', '』', '......。']
['[SEP]'] ['。', '」', '、', '(', ':', 'し', 'は', 'て', '"。', 'た']
